In [6]:
from pathlib import Path
import sys
import numpy as np
from collections import defaultdict



from rl_envs.gym_grid_world_env import GridWorldEnv
from tools.helper import *
from agents.Q_Learning import QLearningAgent as AgentAlgo


%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2


In [7]:

env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=-1, hit_wall_reward=-1, target_reward=1)
agent = AgentAlgo(action_space_n=int(env.action_n), epsilon=0)

In [11]:
episode_len = 100000
# collecting experiences
obs, _ = env.reset()
trajectory = []
for _ in range(episode_len):
    state = tuple(obs['agent'])
    action = agent.get_behavior_action(state)
    obs, reward, terminated , truncated, info = env.step(action)
    next_state = tuple(obs['agent'])
    trajectory.append((state, action, reward , next_state))
    state = next_state
"""
注意这里 policy 虽然对了, 但是 state_value 其实还没有收敛. 

循环执行 100 次后能够收敛, 但为了减少执行时间, 将其去除 (若是更复杂的网络或许需要这样)
# for _ in range(100): 
"""
for state, action, reward, next_state in trajectory:
    td_target = reward + 0.9 * max(agent.q[next_state])
    agent.td_learn(state, action, td_target)
    agent.policy_improvement(state)


debug_v = \
[ 1.49 1.08 1.74 1.96 1.44 ]\
[ 2.04 2.83 3.64 2.71 2.05 ]\
[ 2.83 3.66 5.67 3.57 2.79 ]\
[ 3.70 5.68 5.71 5.63 3.60 ]\
[ 2.86 4.77 5.69 4.67 3.71 ]

In [12]:
"""
output
"""
print(env)

print_actions(agent, env, get_optimal = True)


Q = agent.q
V = {}
for state in Q.keys():
    V[state] = max(Q[state])
print("state value generated by algorithm")
print_by_dict(env, V)

print()
V, iteration_count = compute_state_value(env.height, env.width, env, agent.policy)
print("policy's state value evaluated by (model-based) Bellman equation ")
print_by_dict(env, V)
print(iteration_count)



<GridWorldEnv instance>
[  ↓   →   ↓   ↓   ↓  ]
[  ↓   ↓   ↓   ↓   ↓  ]
[  ↓   ↓   ↓   ↓   ↓  ]
[  →   →   ↺   ←   ←  ]
[  ↑   →   ↑   ←   ←  ]
[ 5.83 5.58 6.20 6.48 5.83 ]
[ 6.48 7.20 8.00 7.20 6.48 ]
[ 7.20 8.00 10.00 8.00 7.20 ]
[ 8.00 10.00 10.00 10.00 8.00 ]
[ 7.20 9.00 10.00 9.00 8.10 ]

[ 5.83 5.58 6.20 6.48 5.83 ]
[ 6.48 7.20 8.00 7.20 6.48 ]
[ 7.20 8.00 10.00 8.00 7.20 ]
[ 8.00 10.00 10.00 10.00 8.00 ]
[ 7.20 9.00 10.00 9.00 8.10 ]
88


In [ ]:
gridworld_demo(agent, forbidden_reward=-1, hit_wall_reward=-1, target_reward=1)
